# Create Function Call Chains
This notebook creates the function call chains (the sequence of functions called by other functions) in NoLCAT based on the Function Call Chains diagram. **The notebook uses the commands for executing cells above and below that appear in the upper left menu of code cells; where to use them is marked in bold text.** Only those function chains containing a selected function are printed to limit the overall output.

In [27]:
from collections import deque
import re

## Graph Cycle Finding Function
Function based on code at https://stackoverflow.com/a/40834276.

In [28]:
def find_cycles(edge_pairs):
    adjacency_matrix = dict()  # Keys are calling functions (start nodes) with values of a list of all functions called (all end nodes)
    for pair in edge_pairs:
        adjacency_matrix[pair[0]] = []
    for pair in edge_pairs:
        adjacency_matrix[pair[0]].append(pair[1])
    
    list_of_cycles = []
    for start_node_and_functions_it_calls in adjacency_matrix:
        tracking = [(start_node_and_functions_it_calls, [])]
        while tracking:
            origin_node, called_nodes = tracking.pop()
            if called_nodes and origin_node == start_node_and_functions_it_calls:
                list_of_cycles.append(called_nodes)
                continue
            if adjacency_matrix.get(origin_node):
                for multiple_degree_called_nodes in adjacency_matrix[origin_node]:
                    if multiple_degree_called_nodes in called_nodes:
                        continue
                    tracking.append((multiple_degree_called_nodes, called_nodes+[multiple_degree_called_nodes]))
    
    return list({tuple(nodes_in_cycle) for nodes_in_cycle in list_of_cycles})

## Cycle Edge Removal Function
Function based on code at ???

In [29]:
# Function that removes an edge based on the cycles found above so no cycles go into the function below

## Function Call Chain Finding Function
Function based on code at https://www.geeksforgeeks.org/find-paths-given-source-destination/#approach-2-using-bfs-o-2v-v-time-and-ov-space; which requires nodes to be assigned numbers.

In [30]:
def findFunctionCallChains(number_of_nodes, edges, start_node, end_node):
    graph = [[] for _ in range(number_of_nodes)]
    allCallChains = []
    q = deque()
    q.append([start_node])  # Initialize queue with the starting call chain

    # Build the graph from edges
    for edge in edges:
        graph[edge[0]].append(edge[1])
    
    while q:
        cc = q.popleft()
        current = cc[-1]

        # If destination is reached, store the call chain
        if current == end_node:
            allCallChains.append(cc)
        
        # Explore all adjacent vertices
        for adj in graph[current]:
            newCallChains = list(cc)
            newCallChains.append(adj)
            q.append(newCallChains)
    
    return allCallChains

## Function Call Chain Creation Procedure

### Get DOT File Contents as String

In [31]:
with open("function_call_chains.dot", encoding="utf-8") as f:
    DOT_file = f.read()

### Extract All Function Names
From a network graph perspective, the functions serve as nodes.

In [32]:
raw_node_names = re.findall(r'\s{8}(?:\w|_)+\s\[', DOT_file)
node_names = []
for name in raw_node_names:
    node_names.append(name[8:-2])
number_of_nodes = len(node_names)

### Extract All Calls
From a network graph perspective, the calls serve as edges.

In [33]:
raw_edge_names = re.findall(r'\s{4}(?:\w|_)+\s->\s(?:\w|_)+', DOT_file)
edge_names = []
for name in raw_edge_names:
    pair = name.split(" -> ")
    edge_names.append(
        (pair[0][4:], pair[1])
    )

### Assign Number Values to Functions
The function for finding call chains requires the nodes to be assigned sequential numbers starting with zero (0); since the numbers will need to be converted back to function names, a reverse lookup is also created.

In [34]:
lookup_by_name = dict()
n = 0

for k in node_names:
    lookup_by_name[k] = n
    n += 1

lookup_by_number = {v: k for k, v in lookup_by_name.items()}

### Create Edge Array with Numbers

In [35]:
edge_pairs = []
for pair in edge_names:
    edge_pairs.append(
        (lookup_by_name[pair[0]], lookup_by_name[pair[1]])
    )

### Find Cycles
In considering `redirect(url_for())` functions a call, recursive calls (called cycles in graph theory) can occur. The function for finding function call chains goes into an infinite loop when there are recursive calls; this identifies the cycles so they can be removed from the list of pairs.

In [ ]:
cycles = find_cycles(edge_pairs)
cycles = list({tuple(sorted(nodes_of_cycle, reverse=True)) for nodes_of_cycle in cycles})  # Reversal makes removal of the less logical redirect as call more likely

### Remove Pairs Causing Cycles

In [ ]:
# Call function removing one of the edges creating the cycle from `edge_pairs`

### View Function Numbers

In [ ]:
for function_name in sorted(node_names, key=str.lower):  # Key needed for case-insensitive sort
    print(f"{lookup_by_name[function_name]}: {function_name}")

### Select Call Chain Common Function
**Perform "Execute Cells Above" on the cell below.** Using the list output above, select the function to be included in all the output call chains; enter the corresponding number in the cell below, **then perform "Execute Cell and Below"** to finish the notebook.

In [10]:
common_node = 69

### Get Possible Start and End Nodes
To find all function call chains with a function requiring the start and end node, a cartesian product would traditionally need to be used, but not all nodes are used as start or end nodes. By not using any functions that don't call another NoLCAT function (e.g. formatting helpers) as a calling/start node or any function called by another NoLCAT function (e.g. tests) as a called/end node, the number of calls to `findFunctionCallChains()` is dramatically reduced, letting the program run faster.

In [11]:
start_nodes = set()
end_nodes = set()

for pair in edge_pairs:
    start_nodes.add(pair[0])
    end_nodes.add(pair[1])

### Find Function Call Chains
A pair of nested loops gets the cartesian product of the `start_nodes` and `end_nodes` sets. Only the call chains containing the `common_node` value are saved.

In [13]:
function_call_chains = []

for start_node in [1,5,100,250]:
    for end_node in end_nodes:
        if start_node == end_node:
            continue
        call_chains = findFunctionCallChains(number_of_nodes, edge_pairs, start_node, end_node)
        for call_chain in call_chains:
            if common_node in call_chain:
                function_call_chains.append(call_chain)

### Remove Call Chains
Some function call chains will be subsets of other call chains; to minimize the amount of output, call chains that exist as subsets of other call chains being written to output aren't included in the output.

In [ ]:
subset_call_chains = set()

for cc1 in function_call_chains:
    for cc2 in function_call_chains:
        if cc1 == cc2:
            continue
        if str(cc1).strip('[]') in str(cc2).strip('[]'):
            subset_call_chains.add(tuple(cc1))

remove = []  # List of lists prevents use of list comprehension
for cc in subset_call_chains:
    remove.append(list(cc))
function_call_chains = [cc for cc in function_call_chains if cc not in remove]

### Output Function Call Chains

In [ ]:
for call_chain in function_call_chains:
    temp = []
    for node in call_chain:
        temp.append(lookup_by_number[node])
    print(" -> ".join(temp))